
# basic thingies



In [1]:
import pandas as pd
import numpy as np
import eli5

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from math import pi
import heapq

import matplotlib.pyplot as plt
from pathlib import Path

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

Using TensorFlow backend.


In [2]:
file="data/train.json"
data = pd.read_json(file, orient='columns')
data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [3]:
ingredients = data['ingredients']
all_ingredients = []
for item in ingredients:
    all_ingredients.extend(item)
    
#This list will need some cleanup. whole lotta cleanup for data visualization
unique_ingredients = list(set(all_ingredients))
# unique_ingredients

print("all_ingredients: " + str(len(all_ingredients)))
print("unique_ingredients: " + str(len(unique_ingredients)))

all_ingredients: 428275
unique_ingredients: 6714



# machine learning

In [4]:
cuisine_list = data['cuisine']
cuisine_compilation = []
for cuisine in cuisine_list:
    cuisine_compilation.append(cuisine)
      
cuis_unique = list(set(cuisine_compilation))

label_encoder_cuis = LabelEncoder()
label_encoder_cuis.fit(cuis_unique)
encoded_cuis = label_encoder_cuis.transform(data['cuisine'])

one_hot_cuis = to_categorical(encoded_cuis)
one_hot_cuis[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [5]:
data["one_hot_cuisine"] = list(one_hot_cuis)
data = data[["cuisine", "one_hot_cuisine", "ingredients"]]


In [6]:
ingredients = data.loc[:,'ingredients']

i_map = {}
i_list = []
counter = 0
for lists in ingredients:
    for items in lists:
        if items not in i_map:
            i_list.append(items)
            i_map[items] = counter
            counter = counter + 1

ingredients_encodings = []
for lists in ingredients:
    encoding = [0]*len(i_map)
    for items in lists:
        encoding[i_map[items]] = 1
    ingredients_encodings.append(encoding)

In [7]:
data["one_hot_ingredients"] = ingredients_encodings
one_hot_cuisine = data['one_hot_cuisine']
one_hot_ingredients = data['one_hot_ingredients']

In [8]:
ing_train, ing_test,cuis_train, cuis_test = train_test_split(one_hot_ingredients, one_hot_cuisine, random_state=1)

cuis_train = np.array(cuis_train.tolist())
cuis_test = np.array(cuis_test.tolist())
ing_train = np.array(ing_train.tolist())
ing_test = np.array(ing_test.tolist())

model = Sequential()
number_inputs = len(i_list)
number_hidden_nodes = 4
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
#unique ingredients
number_classes = 6714
model.add(Dense(units=number_classes, activation='softmax'))

#unique cuisines
number_classes = 20
model.add(Dense(units=number_classes, activation='softmax'))

In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 26860     
_________________________________________________________________
dense_1 (Dense)              (None, 6714)              33570     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                134300    
Total params: 194,730
Trainable params: 194,730
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(
    ing_train,
    cuis_train,
    epochs=20,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 7s - loss: 2.5987 - acc: 0.2078
Epoch 2/20
 - 7s - loss: 2.2812 - acc: 0.2622
Epoch 3/20
 - 7s - loss: 2.0390 - acc: 0.3260
Epoch 4/20
 - 8s - loss: 1.8567 - acc: 0.3456
Epoch 5/20
 - 8s - loss: 1.7808 - acc: 0.3506
Epoch 6/20
 - 8s - loss: 1.7305 - acc: 0.3602
Epoch 7/20
 - 8s - loss: 1.6830 - acc: 0.4047
Epoch 8/20
 - 8s - loss: 1.6125 - acc: 0.4545
Epoch 9/20
 - 8s - loss: 1.5275 - acc: 0.4956
Epoch 10/20
 - 8s - loss: 1.4594 - acc: 0.5217
Epoch 11/20
 - 8s - loss: 1.3949 - acc: 0.5543
Epoch 12/20
 - 7s - loss: 1.3342 - acc: 0.5752
Epoch 13/20
 - 8s - loss: 1.2833 - acc: 0.5930
Epoch 14/20
 - 9s - loss: 1.2366 - acc: 0.6102
Epoch 15/20
 - 9s - loss: 1.1924 - acc: 0.6272
Epoch 16/20
 - 8s - loss: 1.1498 - acc: 0.6463
Epoch 17/20
 - 8s - loss: 1.1101 - acc: 0.6591
Epoch 18/20
 - 8s - loss: 1.0716 - acc: 0.6716
Epoch 19/20
 - 8s - loss: 1.0358 - acc: 0.6790
Epoch 20/20
 - 8s - loss: 1.0008 - acc: 0.6901


# ELI5 TESTING

In [12]:
# from xgboost import XGBClassifier
# from sklearn.feature_extraction import DictVectorizer
# from sklearn.pipeline import make_pipeline
# from sklearn.model_selection import cross_val_score

# clf = XGBClassifier()
# vec = DictVectorizer()
# pipeline = make_pipeline(vec, clf)

# def evaluate(_clf):
#     scores = cross_val_score(_clf, all_xs, all_ys, scoring='accuracy', cv=10)
#     print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
#     _clf.fit(train_xs, train_ys)  # so that parts of the original pipeline are fitted

# evaluate(pipeline)

NameError: name 'all_xs' is not defined

In [15]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(data, model.predict_proba)
te.show_prediction(target_names=twenty_train.target_names)

AttributeError: 'DataFrame' object has no attribute 'lower'